In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
# from fastai.text import *   # Quick accesss to NLP functionality
import html
from pathlib import Path

In [3]:
import torch.nn as nn

In [4]:
from fastai.text.models.transformer import tfmer_lm_config, GeLU, init_transformer
from fastai.text.models.awd_lstm import RNNDropout
from fastai.text.learner import LanguageLearner

In [5]:
PATH=Path.home()/'data/wikitext-2-raw'

In [6]:
from fastai.basic_data import *
from fastai.torch_core import *
from fastai.layers import *

In [7]:
from fastai.text.models.transformer import *

## Bert task - dataloading

In [8]:
bptt = 256
data = load_data(PATH, bs=8, bptt=bptt)

In [9]:
MASK = 'xxmask'

In [10]:
vocab = data.vocab

In [11]:
vocab.itos.append(MASK)

In [12]:
data.show_batch()

idx,text
0,"team of writers handled the script . xxmaj the game 's opening theme was sung by xxmaj may ' n . xxbos xxmaj it met with positive sales in xxmaj japan , and was praised by both xxmaj japanese and western critics . xxmaj after release , it received downloadable content , along with an expanded edition in xxmaj november of that year . xxmaj it was also adapted into"
1,", which relentlessly and brutally and xxunk keeps these vicious , murderous wars . xxmaj it is a vandal state . xxmaj there is a xxmaj russian writer who once described vandal states as xxmaj genghis xxmaj khan with a telegraph . xxmaj israel is xxmaj genghis xxmaj khan with a computer . i feel no emotion of affinity with that state . i have some good friends and their"
2,the 5th xxmaj division to envelop xxup un troops and push them back to xxmaj pusan . xxmaj the 766th was not reinforced ; xxmaj north xxmaj korean planners intended it to move unseen around the xxup un lines while the majority of the xxup un and xxmaj north xxmaj korean troops were locked in fighting around xxmaj taegu and the xxmaj xxunk xxmaj bulge . xxbos xxmaj by this
3,"at least two "" facilitating subjects "" ; the average point score per qualification was xxunk , equating to a xxup c- grade , and the average point score per student was xxunk . xxmaj the xxmaj sunday xxmaj times ranked xxmaj carre 's 101st ( 49th amongst state schools ) in the xxmaj midlands and 750th nationally based on a - xxmaj level and xxup gcse performance in 2012"
4,"mfume , the president of the xxmaj national xxmaj association for the xxmaj advancement of xxmaj colored xxmaj people ( xxup naacp ) , would run . xxmaj mfume had previously served on the xxmaj baltimore xxmaj city xxmaj council and in the xxmaj united xxmaj states xxmaj house of xxmaj representatives . xxmaj schmoke called the race "" his to lose "" . xxmaj however , xxmaj mfume lived"


In [13]:
data.one_batch()

(tensor([[    9,  3531,  5611,  ...,  1105,   622,    29],
         [16077,    19,  4075,  ..., 17572,    60,     9],
         [   12,     9,     6,  ...,   344,    12,     5],
         ...,
         [    9, 13634,   749,  ...,    54,    16,     5],
         [ 1228,   367,   785,  ...,    16,  2218,    12],
         [   15,  4549,     9,  ...,    21,  1107,   869]]),
 tensor([[ 3531,  5611,  3398,  ...,   622,    29,     6],
         [   19,  4075,    16,  ...,    60,     9,   465],
         [    9,     6,  6630,  ...,    12,     5,   113],
         ...,
         [13634,   749,    23,  ...,    16,     5,   307],
         [  367,   785,   321,  ...,  2218,    12,  1636],
         [ 4549,     9, 17780,  ...,  1107,   869,    11]]))

In [14]:
word_range = (0, len(data.vocab.itos))
from fastai.text.transform import *
pad_idx = data.vocab.stoi[PAD]
mask_idx = data.vocab.stoi[MASK]
def mask_tfm(b, word_range=word_range, pad_idx=pad_idx, 
             mask_idx=mask_idx, p=0.2):
    # p = replacement probability
    x,y_lm = b
    x,y_msk = x.clone(),x.clone() # x, x
#     x,y = x.clone(),y.clone()
    rand = torch.rand(x.shape, device=x.device)
    y_msk[rand > p] = pad_idx
    x[rand <= (p*.8)] = mask_idx # 80% = mask
    wrong_word = (rand > (p*.8)) & (rand <= (p*.9)) # 10% = wrong word
    x[wrong_word] = torch.randint(*word_range, [wrong_word.sum().item()], device=x.device)
    return x, (y_msk, y_lm)

In [15]:
def nw_tfm(b, pad_idx=pad_idx, 
             mask_idx=mask_idx, p=0.2):
    # p = replacement probability
    x,y_lm = b
    
    y_msk = y_lm.clone() # x, x
    rand = torch.rand(x.shape, device=x.device)
    y_msk[rand > p] = pad_idx
    x_msk = rand <= p
    return (x, x_msk), (y_msk, y_lm)

In [16]:
# data.train_dl.add_tfm(mask_tfm)
# data.valid_dl.add_tfm(mask_tfm)

In [17]:
data.train_dl.add_tfm(nw_tfm)
data.valid_dl.add_tfm(nw_tfm)

In [18]:
xb,yb = data.one_batch()

In [19]:
# model = get_language_model(TransformerXL, len(data.vocab.itos))

In [20]:
# vocab_sz = len(data.vocab.itos)
# # config = tfmer_lm_config.copy(); config
# config = {
#     'ctx_len': bptt,
#     'n_layers': 4,
#     'n_heads': 4,
#     'd_model': 128,
#     'd_head': 32,
#     'd_inner': 512,
# }

# embed = Embedder(vocab_sz, config['d_model'])
# bert_encoder = Encoder(**config)
# nw_encoder = Encoder(**config)
# decoder = Decoder(vocab_sz, config['d_model'], tie_encoder=embed.embed, bias=False, output_p=False)
# bert_head = BertHead(embed, bert_encoder, nw_encoder, decoder)
# model = bert_head
# model.apply(init_transformer);

# embed(torch.tensor(mask_idx))

# e_out = embed(xb[0])
# nw_out = nw_encoder(e_out)

# nw_out.shape

# xb[1].shape

# xb[1][0][0] = 1

# nw_out[xb[1]] = embed(torch.tensor(mask_idx, device=xb[0].device))

# nw_out

# xb[1]

## Model creation

In [17]:

class Embedder(nn.Module):
    "Embedding + positional encoding + dropout"
    def __init__(self, vocab_sz:int, emb_sz:int, embed_p:float=0., pad_idx=pad_idx):
        super().__init__()
        self.emb_sz = emb_sz
        
        self.embed = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_idx)
        # See https://arxiv.org/abs/1711.09160
        with torch.no_grad(): trunc_normal_(self.embed.weight, std=0.01)
        self.drop = nn.Dropout(embed_p)
    
    def forward(self, inp, pos_forward=False):
        emb = self.drop(self.embed(inp))
        return emb


In [18]:

class Encoder(nn.Module):
    "TransformerXL model: https://arxiv.org/abs/1901.02860."
    def __init__(self, n_layers:int, n_heads:int, d_model:int, d_head:int, d_inner:int, 
                 resid_p:float=0., attn_p:float=0., ff_p:float=0., bias:bool=False, scale:bool=True,
                 act:Activation=Activation.ReLU, double_drop:bool=True, attn_cls:Callable=MultiHeadRelativeAttention,
                 learned_pos_enc:bool=False, mask:bool=True, mem_len:int=0, **kwargs):
        super().__init__()
        self.u = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.v = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.mem_len,self.n_layers,self.d_model,self.mask = mem_len,n_layers,d_model,mask
        self.init = False
        self.layers = nn.ModuleList([DecoderLayer(n_heads, d_model, d_head, d_inner, resid_p=resid_p, attn_p=attn_p,
                      ff_p=ff_p, bias=bias, scale=scale, act=act, double_drop=double_drop, 
                      attn_cls=attn_cls) for k in range(n_layers)])
        
        self.pos_enc = PositionalEncoding(d_model)
    
    def reset(self):
        "Reset the internal memory."
        self.hidden = [next(self.parameters()).data.new(0) for i in range(self.n_layers+1)]

    def _update_mems(self, hids):
        if not getattr(self, 'hidden', False): return None
        assert len(hids) == len(self.hidden), 'len(hids) != len(self.hidden)'
        with torch.no_grad():
            for i in range(len(hids)):
                cat = torch.cat([self.hidden[i], hids[i]], dim=1)
                self.hidden[i] = cat[:,-self.mem_len:].detach()
    
    def select_hidden(self, idxs): self.hidden = [h[idxs] for h in self.hidden]
    
    def forward(self, x):
        #The hidden state has to be initiliazed in the forward pass for nn.DataParallel
        if self.mem_len > 0 and not self.init: 
            self.reset()
            self.init = True
        bs,x_len,emb_sz = x.size()
        
        inp = x
        
        m_len = self.hidden[0].size(1) if hasattr(self, 'hidden') and len(self.hidden[0].size()) > 1 else 0
        seq_len = m_len + x_len
        mask = torch.triu(x.new_ones(x_len, seq_len).long(), diagonal=1+m_len).byte()[None,None] if self.mask else None
        
        hids = []
        pos = torch.arange(seq_len-1, -1, -1, device=inp.device, dtype=inp.dtype)
        pos_enc = self.pos_enc(pos)
        hids.append(inp)
        for i, layer in enumerate(self.layers):
            mem = self.hidden[i] if self.mem_len > 0 else None
            inp = layer(inp, r=pos_enc, u=self.u, v=self.v, mask=mask, mem=mem)
            hids.append(inp)
        core_out = inp[:,-x_len:]
        if self.mem_len > 0 : self._update_mems(hids)
        return core_out

In [19]:
class BertHead(nn.Module):
    def __init__(self, embed, bert_encoder, nw_encoder, decoder, lm_only=False):
        super().__init__()
        self.embed = embed
        self.bert_encoder = bert_encoder
        self.nw_encoder = nw_encoder
        self.decoder = decoder
        self.lm_only = lm_only
        
    
    def forward(self, x, mask_idxs=None):
        
        x_enc = self.embed(x)
#         x_enc, _, _ = self.embed(x)
        if self.lm_only:
            nw_enc = self.nw_encoder(x_enc)
            return self.decoder(nw_enc)
        bert_first = mask_idxs is None # mask idxs tells us which embeddings to mask
        if bert_first:
            bert_enc = self.bert_encoder(x_enc)
            nw_enc = self.nw_encoder(bert_enc)
            return self.decoder(bert_enc), self.decoder(nw_enc)
        else:
            nw_enc = self.nw_encoder(x_enc)
            nw_enc[mask_idxs] = embed(torch.tensor(mask_idx, device=x.device))
            bert_enc = self.bert_encoder(nw_enc)
            return self.decoder(bert_enc), self.decoder(nw_enc)
    
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()
        

In [20]:

class Decoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."
    initrange=0.1

    def __init__(self, n_out:int, n_hid:int, output_p:float, tie_encoder:nn.Module=None, bias:bool=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:
        output = self.output_dp(input)
        decoded = self.decoder(output)
        return decoded

## Transformer

In [21]:
vocab_sz = len(data.vocab.itos)
# config = tfmer_lm_config.copy(); config
config = {
    'ctx_len': bptt,
    'n_layers': 4,
    'n_heads': 4,
    'd_model': 128,
    'd_head': 32,
    'd_inner': 512,
}

In [22]:
class BertLoss():
    def __init__(self, pad_idx=pad_idx):
        "Loss mult - mask, NextWord, Seq2Seq, NextSent"
        self.index_loss = CrossEntropyFlat(ignore_index=pad_idx)
        
    def __call__(self, input:Tensor, lm_target:Tensor, bert_target:Tensor, **kwargs)->Rank0Tensor:
        x_bert, x_lm = input
        loss_bert = self.index_loss.__call__(x_bert, bert_target, **kwargs)
        loss_lm = self.index_loss.__call__(x_lm, lm_target, **kwargs)
#         print(loss_bert, loss_lm)
        return loss_bert + loss_lm

In [23]:
# class BertLearner(LanguageLearner):
    

In [24]:

def acc_ignore_pad(input:Tensor, targ:Tensor, pad_idx=pad_idx)->Rank0Tensor:
    n = targ.shape[0]
    input = input.argmax(dim=-1).view(n,-1)
    targ = targ.view(n,-1)
    mask = targ != pad_idx
    return (input[mask]==targ[mask]).float().mean()

def bert_acc(input:Tensor, t1:Tensor, t2:Tensor)->Rank0Tensor:
    return acc_ignore_pad(input[0], t1)
def lm_acc(input:Tensor, t1:Tensor, t2:Tensor)->Rank0Tensor:
    return acc_ignore_pad(input[1], t1)

In [30]:
embed = Embedder(vocab_sz, config['d_model'])
bert_encoder = Encoder(**config)
nw_encoder = Encoder(**config)
decoder = Decoder(vocab_sz, config['d_model'], tie_encoder=embed.embed, bias=False, output_p=False)
bert_head = BertHead(embed, bert_encoder, nw_encoder, decoder)
model = bert_head
model.apply(init_transformer);

In [31]:
learn = LanguageLearner(data, model, loss_func=BertLoss())

In [32]:
learn.callbacks = []

In [33]:
learn.metrics=[bert_acc, lm_acc]

In [34]:
xb,yb = data.one_batch(cpu=False)

In [37]:
out = model(*xb)

In [38]:
out[0][0].shape

torch.Size([256, 39881])

In [39]:
learn.loss_func(out, *yb)

tensor(21.1966, device='cuda:0', grad_fn=<AddBackward0>)

In [40]:
learn.validate()

[21.195696, tensor(0.0070), tensor(0.0342)]

In [41]:
learn.pred_batch(batch=(xb,yb))

[tensor([[[-4.9580e-01, -9.6619e-02, -5.2701e-01,  ..., -4.7588e-02,
           -1.3230e-01,  3.7609e-01],
          [ 1.5931e+00, -4.1648e-02,  2.2567e-01,  ..., -1.5223e-01,
           -2.6749e-01,  1.7577e-02],
          [ 1.8046e-01,  1.4072e-01,  1.5947e-03,  ..., -3.6805e-01,
            2.7003e-01,  2.6057e-02],
          ...,
          [ 7.7676e-01, -9.8936e-02,  4.2904e-01,  ..., -1.6872e-01,
           -3.4325e-01,  1.0617e-01],
          [ 2.9330e-01,  8.7012e-02,  4.0908e-01,  ..., -1.4277e-01,
           -6.7441e-02,  7.8620e-02],
          [-3.1922e-02,  2.7619e-01,  4.4806e-02,  ..., -2.3188e-01,
            7.5024e-01, -1.3021e-01]],
 
         [[ 1.5724e-01,  2.2869e-01, -3.8199e-01,  ..., -2.6260e-01,
           -3.5921e-01,  5.0073e-01],
          [-6.6347e-03,  1.1319e-01,  1.2132e-02,  ..., -1.6496e-01,
           -3.0566e-01,  5.1331e-02],
          [-1.1949e-01,  1.8414e-01, -2.1055e-02,  ..., -2.6595e-01,
            7.7257e-01, -8.9324e-02],
          ...,
    

In [42]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,bert_acc,lm_acc,time
0,17.509533,17.347189,0.139117,0.035498,01:26


## Baseline

In [26]:
embed = Embedder(vocab_sz, config['d_model'])
bert_encoder = Encoder(**config)
nw_encoder = Encoder(**config)
decoder = Decoder(vocab_sz, config['d_model'], tie_encoder=embed.embed, bias=False, output_p=False)
bert_head = BertHead(embed, bert_encoder, nw_encoder, decoder)
model = bert_head
model.apply(init_transformer);

In [36]:
def lm_tfm(b):
    x,y = b
    return x, (y,)

In [37]:
data.train_dl.add_tfm(lm_tfm)
data.valid_dl.add_tfm(lm_tfm)

In [38]:
learn = LanguageLearner(data, model)

In [39]:
learn.callbacks = []

In [40]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


TypeError: __call__() takes 3 positional arguments but 4 were given

In [41]:
%debug

> /home/ubuntu/fastai/fastai/basic_train.py(30)loss_batch()
     28 
     29     if not loss_func: return to_detach(out), yb[0].detach()
---> 30     loss = loss_func(out, *yb)
     31 
     32     if opt is not None:

ipdb> yb
(tensor([[   1,    1,    1,  ..., 7180,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    5,  ...,    1,    1,    1],
        ...,
        [   1,    1,    1,  ...,    5,    1,    1],
        [   1,    1,  512,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0'), tensor([[    5, 25176,  2490,  ...,  7180,     5,  4539],
        [    9,   299,   141,  ...,   335,    12,     9],
        [   68,    11,     5,  ...,   695,    14,     5],
        ...,
        [    9,     5,   288,  ...,     5,     9,     5],
        [19215,   188,   512,  ...,    15,     5, 21621],
        [   11,     6,  3858,  ...,  2704,   912,    17]], device='cuda:0'))
ipdb> quit


In [30]:
xb,yb = data.one_batch()